# Lab 12-3 many to many fixed
### hihello training 
* many to many
* fixed input sequence length

In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.utils import to_categorical
from pprint import pprint

print(tf.__version__)
tf.enable_eager_execution()

1.12.0


### Prepairing dataset
hih -> ihe, ihe -> hel, hel -> ell, ell -> llo

In [2]:
strings = 'hihello'
char_set = sorted(list(set('hihello' + 'hello')))
idx2char = {idx : char for idx, char in enumerate(char_set)}
char2idx = {char : idx for idx, char in enumerate(char_set)}

In [3]:
data = []
for i in range(len(strings) -2):
    data.append(strings[i:i+3])

x_strings = data[0:4]
y_strings = data[1:5]

print(x_strings, y_strings)

['hih', 'ihe', 'hel', 'ell'] ['ihe', 'hel', 'ell', 'llo']


In [4]:
x_data = np.array(list(map(lambda strings : [char2idx.get(char) for char in strings], x_strings)))
x_data = x_data.astype(np.int32)
y_data = list(map(lambda strings : [char2idx.get(char) for char in strings], y_strings))
y_data = np.array(list(map(lambda data : [to_categorical(token, len(char2idx))for token in data],y_data)))
y_data = y_data.astype(np.int32)
# data 형태확인
print(x_data)
print(y_data)

[[1 2 1]
 [2 1 0]
 [1 0 3]
 [0 3 3]]
[[[0 0 1 0 0]
  [0 1 0 0 0]
  [1 0 0 0 0]]

 [[0 1 0 0 0]
  [1 0 0 0 0]
  [0 0 0 1 0]]

 [[1 0 0 0 0]
  [0 0 0 1 0]
  [0 0 0 1 0]]

 [[0 0 0 1 0]
  [0 0 0 1 0]
  [0 0 0 0 1]]]


In [5]:
# input
tr_dataset = tf.data.Dataset.from_tensor_slices((x_data, y_data))
tr_dataset = tr_dataset.shuffle(buffer_size=4)
tr_dataset = tr_dataset.batch(batch_size = 2)

print(tr_dataset)

<BatchDataset shapes: ((?, 3), (?, 3, 5)), types: (tf.int32, tf.int32)>


### Creating model

In [6]:
num_classes = len(char2idx)
hidden_dim = 10
max_sequence = 3

input_dim = len(char2idx)
output_dim = len(char2idx)
one_hot = np.eye(len(char2idx))

model = keras.Sequential()
model.add(keras.layers.InputLayer(input_shape=(max_sequence,)))
model.add(keras.layers.Embedding(input_dim=input_dim, output_dim=output_dim,
                                 trainable=False, input_length=max_sequence,
                                 embeddings_initializer=keras.initializers.Constant(one_hot)))
model.add(keras.layers.SimpleRNN(units=hidden_dim, return_sequences=True))
model.add(keras.layers.TimeDistributed(keras.layers.Dense(units=num_classes)))

In [7]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 3, 5)              25        
_________________________________________________________________
simple_rnn (SimpleRNN)       (None, 3, 10)             160       
_________________________________________________________________
time_distributed (TimeDistri (None, 3, 5)              55        
Total params: 240
Trainable params: 215
Non-trainable params: 25
_________________________________________________________________


### Training model

In [8]:
def loss_fn(model, x, y):
    return tf.losses.softmax_cross_entropy(onehot_labels=y, logits=model(x))

lr = 0.1
epochs = 30
opt = tf.train.AdamOptimizer(learning_rate = lr)

In [9]:
for epoch in range(epochs):
    avg_tr_loss = 0
    tr_step = 0
    
    for x_mb, y_mb in tr_dataset:
        with tf.GradientTape() as tape:
            tr_loss = loss_fn(model, x=x_mb, y=y_mb)
        grads = tape.gradient(target=tr_loss, sources=model.variables)
        opt.apply_gradients(grads_and_vars=zip(grads, model.variables))
        avg_tr_loss += tr_loss
        tr_step += 1
    else:
        avg_tr_loss /= tr_step
    
    print('epoch : {:3}, tr_loss : {:.3f}'.format(epoch + 1, avg_tr_loss))

epoch :   1, tr_loss : 1.830
epoch :   2, tr_loss : 0.926
epoch :   3, tr_loss : 0.609
epoch :   4, tr_loss : 0.392
epoch :   5, tr_loss : 0.257
epoch :   6, tr_loss : 0.183
epoch :   7, tr_loss : 0.179
epoch :   8, tr_loss : 0.162
epoch :   9, tr_loss : 0.142
epoch :  10, tr_loss : 0.149
epoch :  11, tr_loss : 0.127
epoch :  12, tr_loss : 0.138
epoch :  13, tr_loss : 0.134
epoch :  14, tr_loss : 0.132
epoch :  15, tr_loss : 0.135
epoch :  16, tr_loss : 0.119
epoch :  17, tr_loss : 0.131
epoch :  18, tr_loss : 0.119
epoch :  19, tr_loss : 0.123
epoch :  20, tr_loss : 0.117
epoch :  21, tr_loss : 0.125
epoch :  22, tr_loss : 0.135
epoch :  23, tr_loss : 0.136
epoch :  24, tr_loss : 0.124
epoch :  25, tr_loss : 0.117
epoch :  26, tr_loss : 0.117
epoch :  27, tr_loss : 0.128
epoch :  28, tr_loss : 0.117
epoch :  29, tr_loss : 0.117
epoch :  30, tr_loss : 0.129


### Checking performance

In [10]:
yhat = model.predict(np.array(x_data))
yhat = np.argmax(yhat, axis=-1)
print(yhat)

[[0 1 0]
 [1 0 3]
 [0 3 3]
 [3 3 4]]


In [11]:
ytrue = np.argmax(y_data, axis = -1)
print(ytrue)

[[2 1 0]
 [1 0 3]
 [0 3 3]
 [3 3 4]]


In [12]:
print('acc : {:.2%}'.format(np.mean(np.apply_along_axis(lambda row: all(row),-1, yhat == ytrue))))

acc : 75.00%
